# Joins and Sets on Dataframe

We will cover:

1. Clean and Encrich Data using Snowpark for Python
    
    a. Perform Joins

2. Perform Aggregate and Set based Operations on Dataframe

    a. Union/Intersect/Except


For more information follow the below links:

1. [Working with DataFrames in Snowpark Python]('https://docs.snowflake.com/en/developer-guide/snowpark/python/working-with-dataframes#joining-dataframes')

2. [Joins in Snowflake Snowpark]('https://thinketl.com/joins-in-snowflake-snowpark/')


In [ ]:
from snowflake.snowpark.context import get_active_session

session =  get_active_session()

In [ ]:
# create two related test dataframes
persons = session.create_dataframe([
    [1, "John", 2], [2, "Mary", None], [3, "Mark", 2]],
    schema=["id", "name", "id_parent"])
persons

data = session.create_dataframe([
    [1, "Teacher", 28], [2, "Engineer", 22], [4, "Architect", 45]],
    schema=["id", "profession", "age"])
data

In [ ]:
#Inner Joins:

persons.join(data,'id')

In [ ]:
#Natural Joins
persons.natural_join(data)

In [ ]:
#Joins with Same columns
persons.join(data,persons.col("id")  == data.col("id"))

In [ ]:
# Renaming columns in Select
persons.join(data,
    persons.col("id") == data.col("id")
    ).select(persons["id"].alias("id_person"), "name",
             data["id"].alias("id_data"), "profession", "age")

In [ ]:
# Overriding directly the auto-prefix
persons.join(data,
           persons.col("id") == data.col("id"),
           lsuffix="_person", rsuffix="_data")

In [ ]:
# Left Outer Join
persons.join(data,persons.col("id") == data.col("id"),how="left")

In [ ]:
# Left Anti Join
persons.join(data,
    persons.col("id") == data.col("id"),
    how="leftanti")

In [ ]:
# Cross Join
persons.join(data, how="cross")

In [ ]:
#Sefl Join
persons.join(persons,
    persons["id"] == persons["id_parent"])

In [ ]:
# fix for self-join
from copy import copy
parents = copy(persons)

parents.join(persons,
    persons["id_parent"] == parents["id"]
    ).select(
        persons["name"].alias("child"),
        parents["name"].alias("parent"))

In [ ]:
# other two test data frames for set operations
employees = session.create_dataframe([
    ["John", 28], ["Mary", 22], ["Mark", 51]],
    schema=["name", "age"])
employees

customers = session.create_dataframe([
    ["John", 28], ["Adele", 18], ["George", 34], ["Mark", 51]],
    schema=["full_name", "customer_age"])
customers

In [ ]:
employees.union(customers)

In [ ]:
employees.union_all(customers)

In [ ]:
# union by name will match sets by their column names
customers2 = customers.select(
    customers["customer_age"].alias("age"),
    customers["full_name"].alias("name"))
employees.union_by_name(customers2)

In [ ]:
employees.intersect(customers)

In [ ]:
#Subtract/Minus/Except
employees.subtract(customers)